## Overture Maps Data Download Test
### Gavin Rolls

NOTE: Use urbsim kernel to avoid parse error

In [1]:
#Library Imports - using DuckDB for Overture Import
import duckdb
import pandas as pd

In [3]:
#Config SQL
%pip install ipython-sql duckdb duckdb-engine jupysql --quiet
%pip install --upgrade grpcio --quiet
%load_ext sql

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [7]:
%%sql      
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

,Success


### Demo Direct Loading of Overture Data (Demo from Data Extraction Docs - Overture Maps)

In [ ]:
#Takes forever - just a proof of concept. It does work, though

# %%sql

# SET s3_region='us-west-2';

# CREATE OR REPLACE VIEW admins_view AS (
#     SELECT
#         *
#     FROM
#         read_parquet('s3://overturemaps-us-west-2/release/2024-05-16-beta.0/theme=admins/type=*/*', filename=true, hive_partitioning=1)
# );

# COPY (
#     SELECT
#            admins.id,
#            admins.subtype,
#            admins.iso_country_code_alpha_2,
#            admins.admin_level,
#            areas.area_id,
#            names.primary AS primary_name,
#            sources[1].dataset AS primary_source,
#            ST_GeomFromWkb(areas.area_geometry) AS geometry
#       FROM admins_view AS admins
#       INNER JOIN (
#            SELECT
#                   id AS area_id,
#                   locality_id,
#                   geometry AS area_geometry
#            FROM admins_view
#        ) AS areas ON areas.locality_id == admins.id
#       WHERE admins.admin_level = 1
#  ) TO 'countries_overture.geojson'
# WITH (FORMAT GDAL, DRIVER 'GeoJSON', SRS 'EPSG:4326');

### Using Apache Sedona to Query Overture

In [9]:
#Sedona Libraries

%pip install apache-sedona --quiet
%pip install pyspark --quiet

from sedona.spark import *
import os
import time

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
DATA_LINK = "s3a://wherobots-examples/data/overturemaps-us-west-2/release/2023-07-26-alpha.0/"

In [17]:
#Sedona Setup
config = SedonaContext.builder()\
    .config("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")\
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")\
    .getOrCreate()

sedona = SedonaContext.create(config)

/Users/gavinrolls/anaconda3/envs/urbsim/lib/python3.11/site-packages/pyspark/bin/spark-class: line 71: /usr/bin/java/bin/java: Not a directory
/Users/gavinrolls/anaconda3/envs/urbsim/lib/python3.11/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.